In [1]:
import glob
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.advanced_activations import LeakyReLU,PReLU
from keras.models import load_model #for saving and loading a model
from keras.callbacks import ModelCheckpoint #for saving  for each epoch
from keras.models import Model
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
from sklearn.metrics import accuracy_score
import scipy.io

AttributeError: module 'keras.utils.generic_utils' has no attribute 'populate_dict_with_module_objects'

In [150]:
def loadFV(folder):
    global views
    global canonical_view
    single_view_descriptors=np.empty(shape=(1,2000))*0.0 
    descriptor_labels=[]
    i=0 #for printing
    for view in views:
            #vectors=[]
            labels=[]
            i=i+1
            folder=data_dir+view         
            #folder_vector=(glob.glob(folder+"/vectors/"+"*.txt"))
            folder_label=(glob.glob(folder+"/labels/"+"*.txt"))
            trainY=open(folder_label[0], "r")
            labels= trainY.readlines()
            labels=labels[0:1240]
            #vectors = np.loadtxt(folder_vector[0])
            #single_view_descriptors=np.vstack((single_view_descriptors,vectors))
            descriptor_labels.extend(labels)
            #if view==canonical_view:
                # canonical_view_descriptors=vectors 
            print("[INFO] processed {}/{}".format(i, 11))
             
         #df_canonical = pd.DataFrame(canonical_view_descriptors,header=None)
         #df_canonical.to_csv("canonical_view.csv")   
             
         
         #df_labels = pd.DataFrame(descriptor_labels,header=None)
         #df_labels.to_csv("labels.csv") 
             
        #single_view_descriptors=np.delete(single_view_descriptors,0,0) #removing first row of zeros
         #df_data = pd.DataFrame(single_view_descriptors,header=None)
         #df_data.to_csv("data.csv")
         
   # else:
        #df_data= pd.read_csv('data.csv')
        #df_canonical= pd.read_csv('canonical_view.csv')
        #df_labels= pd.read_csv('labels.csv')
    #if (vectors) and (labels):del vectors,labels
    return descriptor_labels
    

 
 
    

def get_activationn(layernum):
    intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.layers[layernum].output) 
    intermediate_output = intermediate_layer_model.predict(test_X)
    return intermediate_output
#def create_crossviewDescriptors(cross_view_layers):
def preparetestData(views,cross_view_descriptors,descriptor_labels,numSamplePerView):
    len_crs_v_des=cross_view_descriptors.shape[1]
    fn_tstData=np.empty((1,len_crs_v_des))*0;fn_tstLabels=[]
    fb_tstData=np.empty((1,len_crs_v_des))*0;fb_tstLabels=[]
    fc_tstData=np.empty((1,len_crs_v_des))*0;fc_tstLabels=[]
    for index in range(0,len(descriptor_labels),numSamplePerView):
        for view in views:
            if (descriptor_labels[index][:3]!=view):
                continue
            # split label by underscore, labels are required to follow the 
            # convention view_label_serial_seq
#            components =descriptor_labels[index].split('_')
#             # assign components to variables for easier access
#            label = int(components[1])
#            seq_no = components[3].strip('\n')
            
            #feature_vector=cross_view_descriptors[index]
            temp_labels=descriptor_labels[index:index+numSamplePerView]
            temp_data=cross_view_descriptors[index:index+numSamplePerView]
            for i in range(0,len(temp_data),10):
                fb_tstData=np.vstack((fb_tstData,temp_data[i:i+2]))
                fc_tstData=np.vstack((fc_tstData,temp_data[i+2:i+4]))
                fn_tstData=np.vstack((fn_tstData,temp_data[i+4:i+10]))
                fb_tstLabels.extend([s.split('_')[1] for s in temp_labels[i:i+2]])
                fc_tstLabels.extend([s.split('_')[1] for s in temp_labels[i+2:i+4]])
                fn_tstLabels.extend([s.split('_')[1] for s in temp_labels[i+4:i+10]])
                print("[INFO] processed {}/{}".format(i, len(temp_data)))
                
            #assign feature vector to training or test
#            if (seq_no=='nm'):
#                fn_tstData=np.vstack((fn_tstData,feature_vector))
#                fn_tstLabels.append(label)
#            if((seq_no=='bg')):
#                fb_tstData=np.vstack((fb_tstData,feature_vector))
#                fb_tstLabels.append(label)
#            if( (seq_no=='cl')):
#                fc_tstData=np.vstack((fc_tstData, feature_vector))
#                fc_tstLabels.append(label)
            break
    del temp_labels,temp_data
    fn_tstData=np.delete(fn_tstData,0,0);fb_tstData=np.delete(fb_tstData,0,0);fc_tstData=np.delete(fc_tstData,0,0)
    return fn_tstData,fn_tstLabels,fb_tstData,fb_tstLabels,fc_tstData,fc_tstLabels

def preparetrainData(views,cross_view_descriptors,descriptor_labels,numSamplePerView):
    i=0 #for printing
    len_crs_v_des=cross_view_descriptors.shape[1]
    tr_Data=np.empty((1,len_crs_v_des))*0;tr_Labels=[]
    for index in range(0,len(descriptor_labels),numSamplePerView):
        for view in views:
            
            if (descriptor_labels[index][:3]!=view):
                continue
            print(view)
            tr_Data=np.vstack((tr_Data,cross_view_descriptors[index:index+numSamplePerView]))
            temp_labels=descriptor_labels[index:index+numSamplePerView]
            temp_labels=[s.split('_')[1] for s in temp_labels]
            tr_Labels.extend(temp_labels)
            print(len(tr_Labels))
            i=i+1
            break
        print("[INFO] processed {}/{}".format(i, len(views)))        
    tr_Data=np.delete(tr_Data,0,0)
    del temp_labels
    return tr_Data,tr_Labels
def PrepareDataNktm(subject,single_view_descriptors,descriptor_labels,indexOfCanon):
    features=single_view_descriptors.shape[1]
    total_len=len(subject)*10*11
    tr_subjects=np.empty((1,features))*0;tr_labels=[]
    target_view=np.empty((1,features))*0;target_labels=[]
    test_subjects=np.empty((1,features))*0;test_labels=[]
    test_subjects_target=np.empty((1,features))*0;test_labels_target=[]
    for index in range(0,len(descriptor_labels),1240):
        print("[INFO] processed {}/{}".format(index, len(descriptor_labels))) 
        for index3 in range(index,index+1240,10):
                if descriptor_labels[index3].split('_')[1] in subject:
                    print(descriptor_labels[index3])
                    tr_subjects=np.vstack((tr_subjects,single_view_descriptors[index3:index3+10]))
                    tr_labels=tr_labels+descriptor_labels[index3:index3+10]
                    if(index==indexOfCanon):
                        target_view=np.vstack((target_view,single_view_descriptors[index3:index3+10]))
                        target_labels=target_labels+descriptor_labels[index3:index3+10]
                else:
                        test_subjects=np.vstack((test_subjects,single_view_descriptors[index3:index3+10]))
                        test_labels=test_labels+descriptor_labels[index3:index3+10]
                        if (index==indexOfCanon):    
                            test_subjects_target=np.vstack((test_subjects_target,single_view_descriptors[index3:index3+10]))
                            test_labels_target=test_labels_target+descriptor_labels[index3:index3+10]

           
    return tr_subjects,tr_labels,target_view,target_labels,test_subjects,test_labels,test_subjects_target,test_labels_target

In [151]:
subject=list(range(1,11)) #list of subjects for training 
subject=[str(i) for i in subject]
print(subject)


['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']


In [152]:
subject=list(range(1,10)) #list of subjects for training

# fix random seed for reproducibility
np.random.seed(7)

    
data_dir="/home/nazish/maryiam/casiaB/"   
output_dir='/home/nazish/maryiam/python code/output/'
views=['000','018','036','054','072','090','108','126','144','162','180']
#views=['000','090']
canonical_view='090'
indexOfCanon=views.index(canonical_view)*1240
cross_view_layers=2 #concatenating outputs of last 2 layers
hiddenLayers=2
totaLayers=hiddenLayers+2
hidden_layers_size=[1000,1000]
subject=[str(i) for i in subject]

single_view_descriptors=np.loadtxt('/home/nazish/maryiam/casiaB/hog_mbhx_mbhy_REPETITION_2_VIEWS_1_2_3_4_5_6.txt')


#canonical_view_descriptors=np.loadtxt('/home/nazish/maryiam/casiaB/090/vectors/Vectors_hog_mbhx_mbhy_VIEWS_090.txt')
#targetData=canonical_view_descriptors
#loading data
#single_view_descriptors,descriptor_labels,canonical_view_descriptors=loadFV(data_dir)


#single_view_descriptors=df_data.as_matrix(columns=None)
#canonical_view_descriptors=df_canonical.as_matrix(columns=None)
#descriptor_labels=df_labels.as_matrix(columns=None)
#del df_data,df_canonical,df_labels

#CREATING THE TARGET DATA BY REPEATING THE CANONICAL MATRIX 11 times!

#targetData=np.tile(canonical_view_descriptors,[len(views),1])




In [153]:
len(single_view_descriptors)

13640

In [154]:
descriptor_labels=loadFV(data_dir)
#labels=open(folder_label[0], "r")
#descriptor_labels= labels.readlines()

[INFO] processed 1/11
[INFO] processed 2/11
[INFO] processed 3/11
[INFO] processed 4/11
[INFO] processed 5/11
[INFO] processed 6/11
[INFO] processed 7/11
[INFO] processed 8/11
[INFO] processed 9/11
[INFO] processed 10/11
[INFO] processed 11/11


In [155]:
print(len(tr_l))
print(len(target_labels))
print(len(train_Y))
print(len(test_X))
print(len(test_label))
print(len(test_Y))
print(len(test_Y_label))

990
90
90
12650
12650
1150
1150


In [156]:
train_X,tr_l,train_Y,target_labels,test_X,test_label,test_Y,test_Y_label=PrepareDataNktm(subject,single_view_descriptors,descriptor_labels,indexOfCanon)

[INFO] processed 0/13640
000_1_1_bg

000_2_1_bg

000_3_1_bg

000_4_1_bg

000_5_1_bg

000_6_1_bg

000_7_1_bg

000_8_1_bg

000_9_1_bg

[INFO] processed 1240/13640
018_1_1_bg

018_2_1_bg

018_3_1_bg

018_4_1_bg

018_5_1_bg

018_6_1_bg

018_7_1_bg

018_8_1_bg

018_9_1_bg

[INFO] processed 2480/13640
036_1_1_bg

036_2_1_bg

036_3_1_bg

036_4_1_bg

036_5_1_bg

036_6_1_bg

036_7_1_bg

036_8_1_bg

036_9_1_bg

[INFO] processed 3720/13640
054_1_1_bg

054_2_1_bg

054_3_1_bg

054_4_1_bg

054_5_1_bg

054_6_1_bg

054_7_1_bg

054_8_1_bg

054_9_1_bg

[INFO] processed 4960/13640
072_1_1_bg

072_2_1_bg

072_3_1_bg

072_4_1_bg

072_5_1_bg

072_6_1_bg

072_7_1_bg

072_8_1_bg

072_9_1_bg

[INFO] processed 6200/13640
090_1_1_bg

090_2_1_bg

090_3_1_bg

090_4_1_bg

090_5_1_bg

090_6_1_bg

090_7_1_bg

090_8_1_bg

090_9_1_bg

[INFO] processed 7440/13640
108_1_1_bg

108_2_1_bg

108_3_1_bg

108_4_1_bg

108_5_1_bg

108_6_1_bg

108_7_1_bg

108_8_1_bg

108_9_1_bg

[INFO] processed 8680/13640
126_1_1_bg

126_2_1_bg


In [157]:
train_Y=np.delete(train_Y,0,0)
train_X=np.delete(train_X,0,0)
test_Y=np.delete(test_Y,0,0)
test_X=np.delete(test_X,0,0)


In [158]:
numSamplePerView=len(test_Y_label)
numSamplePerView

1150

In [159]:
targetData=np.tile(train_Y,[len(views),1])
targetDataTest=np.tile(test_Y,[len(views),1])

In [160]:
#%%
#returns a compiled model
model_name='model1'
model_path =glob.glob('models/models/'+model_name+'*.hdf5')

if (model_path):
    model = load_model(model_path[0])
else:
    # create model
    model = Sequential()
    model.add(Dense(hidden_layers_size[0], kernel_initializer='uniform',input_dim=desc_shape[1],name="layer_1"))
    model.add(LeakyReLU())
    model.add(Dense(hidden_layers_size[1],kernel_initializer='uniform',name="layer_2"))
    model.add(LeakyReLU())
    model.add(Dense(desc_shape[1], activation='linear',name="output_layer"))

    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam',metrics=['mse'])
    
    # Set callback functions to early stop training and save the best model so far(for now only saving model after each epoch)
    checkpoint = [ModelCheckpoint(filepath='models/intermediate_models/modelintermediate.hdf5')]
    
    # Fit the model
    #model.fit(single_view_descriptors, targetData, epochs=5,callbacks=checkpoint,verbose=1,batch_size=1000) 
    model.fit(train_X, targetData, epochs=5,callbacks=checkpoint,verbose=1,batch_size=500) 
    model.save('models/nktm_model/model1.hdf5')
# evaluate the model
#scores = model.evaluate(single_view_descriptors,targetData,batch_size=1000)
scores = model.evaluate(test_X,targetDataTest,batch_size=1000)

print("loss=%.2f metric_loss=%.2f%%" % (scores[0],scores[1]))


Epoch 1/5
990/990 [==============================] - 0s - loss: 0.0747 - mean_squared_error: 0.0747     
Epoch 2/5
990/990 [==============================] - 0s - loss: 0.0568 - mean_squared_error: 0.0568     
Epoch 3/5
990/990 [==============================] - 0s - loss: 0.0461 - mean_squared_error: 0.0461     
Epoch 4/5
990/990 [==============================] - 0s - loss: 0.0379 - mean_squared_error: 0.0379     
Epoch 5/5
12650/12650 [==============================] - 1s     
loss=0.05 metric_loss=0.05%


In [164]:
desc_shape=test_X.shape
desc_shape

(12650, 2000)

In [165]:
#%%
#getting outputs of intermediate layers:
#outputs = [layer.output for layer in model.layers]  
print("constructing cross view descriptors")
cross_view_descriptors=np.empty(shape=(desc_shape[0],1))*0.0  
layer_nums=[3,4]
#[layer_nums.append(totaLayers-i-1)for i in reversed(range(hiddenLayers))]
for layerNum in layer_nums:
    cross_view_descriptors=np.hstack((cross_view_descriptors,get_activationn(layerNum)))
    print(layerNum)
cross_view_descriptors=np.delete(cross_view_descriptors,0,1)


constructing cross view descriptors
4


In [ ]:
#%%preparing training data
accuracies=dict.fromkeys(views)
for svm_view in views:
    train_views=views[:]
    train_views.remove(svm_view)
    model_svm_name='SVM_model_'+str(svm_view)+'.pkl'
    model_svm_path ='models/svm_models/'+model_svm_name+'.pkl'

    if model_svm_name==model_svm_path[-17:]:
        clf = joblib.load(model_svm_path) 
    else:
        print('PREPARING TRAINING DATA')
        tr_data,tr_labels=preparetrainData(train_views,cross_view_descriptors,test_label,numSamplePerView)
    #del cross_view_descriptors
        print("Fitting the classifier to the training set")
        param_grid = {'C': [5e3, 1e4, 5e],
             'gamma': [0.0005, 0.001, 0.005, 0.1], }
        #param_grid={'C':[1e4],'gamma':[0.001],}
        clf = GridSearchCV(svm.SVC(kernel='rbf', class_weight='balanced',max_iter=-1,verbose=True,decision_function_shape='ovo'), param_grid)
   #     clf=svm.SVC(kernel='rbf', class_weight='balanced',max_iter=10,verbose=True,decision_function_shape='ovo')
        clf = clf.fit(tr_data, tr_labels)
        model_svm_path ='models/svm_models/'+model_svm_name+'.pkl'
        joblib.dump(clf, model_svm_path)
    
#print("Best estimator found by grid search:")
#print(clf.best_estimator_)
#%%
    
    print('PREPARING TEST DATA')
    acc=[]

    fn_tstData,fn_tstLabels,fb_tstData,fb_tstLabels,fc_tstData,fc_tstLabels=preparetestData([svm_view],cross_view_descriptors,test_label,numSamplePerView)
#%%
    #Classification
    print('--- Performing classification with angle %s ---\n'%(svm_view))

    print('--- Performing classification of normal test data')
    y_pred = clf.predict(fn_tstData)
    acc.append(accuracy_score(fn_tstLabels, y_pred))
    print('--- Performing classification of test data with bag')
    y_pred = clf.predict(fb_tstData)
    acc.append(accuracy_score(fb_tstLabels, y_pred))
    print('--- Performing classification of test data with cloak')
    y_pred = clf.predict(fc_tstData)
    acc.append(accuracy_score(fc_tstLabels, y_pred))
    accuracies[svm_view]=acc
    
    print('nm_accuracy:%s , fb_accuracy:%s , fc_accuracy:%s \n' %(acc[0],acc[1],acc[2]))


PREPARING TRAINING DATA
[INFO] processed 0/10
018
1150
[INFO] processed 1/10
036
2300
[INFO] processed 2/10
054
3450
[INFO] processed 3/10
072
4600
[INFO] processed 4/10
090
5750
[INFO] processed 5/10
108
6900
[INFO] processed 6/10
126
8050
[INFO] processed 7/10
144
9200
[INFO] processed 8/10
162
10350
[INFO] processed 9/10
180
11500
[INFO] processed 10/10
Fitting the classifier to the training set
[LibSVM]

/usr/local/lib/python3.5/dist-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=10).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


[LibSVM]

/usr/local/lib/python3.5/dist-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=10).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


[LibSVM]

...

/usr/local/lib/python3.5/dist-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=10).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


[LibSVM]

/usr/local/lib/python3.5/dist-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=10).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


[LibSVM]

/usr/local/lib/python3.5/dist-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=10).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [ ]:
accuracies_df=pd.DataFrame(accuracies, index=['normal', 'bag', 'cloak'])
accuracies_df.to_csv("output.csv")